In [40]:
# Importar librerías
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mlxtend.preprocessing import standardize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

def func_eval(fname, x):
    match fname:
        case "purelin":
            y = x
        case "logsig":
            y = 1.0 / (1.0 + np.exp(-x))
        case "tansig":
            y = 2.0 / (1.0 + np.exp(-2.0 * x)) - 1.0
    return y

def deriv_eval(fname, y):  # Atención que y es la entrada y=f( x )
    match fname:
        case "purelin":
            d = 1.0
        case "logsig":
            d = y * (1.0 - y)
        case "tansig":
            d = 1.0 - y * y
    return d

# Vectorizar la función para poder pasarle un vector para los cálculos
func_eval_vec = np.vectorize(func_eval)

# Vectorizar la función para poder pasarle un vector para los cálculos
deriv_eval_vec = np.vectorize(deriv_eval)

entrada = np.load("x_pca_reduc.npy")
X = np.array(entrada)

# Lectura de archivo con nombres (var respuesta)
nombres = np.load("nombres.npy")

# Convertir los valores de la var categórica en números
Y = LabelBinarizer().fit_transform(nombres)

# Definición de parámetros de la red neuronal
filas_qty = len(X)
input_size = X.shape[1]  # 2 entradas
hidden_size1 = 10  # neuronas en la primera capa oculta
hidden_size2 = 7    # neuronas en la segunda capa oculta
output_size = Y.shape[1]  # neuronas de salida

# Definir las funciones de activación de cada capa
hidden_FUNC1 = 'logsig'  # usamos la logistica para la primera capa oculta
hidden_FUNC2 = 'logsig'  # usamos la logistica para la segunda capa oculta
output_FUNC = 'logsig'  # usamos la logistica para la salida

# Incializar las matrices de pesos azarosamente
np.random.seed(1021) # Usamos la querida random seed Denicolay para que las corridas sean reproducibles
W1 = np.random.uniform(-0.5, 0.5, [hidden_size1, input_size])
X01 = np.random.uniform(-0.5, 0.5, [hidden_size1, 1])
W2 = np.random.uniform(-0.5, 0.5, [hidden_size2, hidden_size1])
X02 = np.random.uniform(-0.5, 0.5, [hidden_size2, 1])
W3 = np.random.uniform(-0.5, 0.5, [output_size, hidden_size2])
X03 = np.random.uniform(-0.5, 0.5, [output_size, 1])

# Avanzar en la red, forward
hidden_estimulos1 = W1 @ X.T + X01
hidden_salidas1 = func_eval_vec(hidden_FUNC1, hidden_estimulos1)
hidden_estimulos2 = W2 @ hidden_salidas1 + X02
hidden_salidas2 = func_eval_vec(hidden_FUNC2, hidden_estimulos2)
output_estimulos = W3 @ hidden_salidas2 + X03
output_salidas = func_eval_vec(output_FUNC, output_estimulos)

# Calcular el error promedio general de TODOS los X
Error = np.mean((Y.T - output_salidas) ** 2)
print(f"Initial error: {Error}")

# Inicializar
epoch_limit = 2000
Error_umbral = 1.0e-06
learning_rate = 0.3
Error_last = 10
epoch = 0

while math.fabs(Error_last - Error) > Error_umbral and epoch < epoch_limit:
    epoch += 1
    Error_last = Error

    for fila in range(filas_qty):
        # Propagar el x hacia adelante
        hidden_estimulos1 = W1 @ X[fila : fila + 1, :].T + X01
        hidden_salidas1 = func_eval_vec(hidden_FUNC1, hidden_estimulos1)
        hidden_estimulos2 = W2 @ hidden_salidas1 + X02
        hidden_salidas2 = func_eval_vec(hidden_FUNC2, hidden_estimulos2)
        output_estimulos = W3 @ hidden_salidas2 + X03
        output_salidas = func_eval_vec(output_FUNC, output_estimulos)

        # Calcular los errores
        ErrorSalida = Y[fila : fila + 1, :].T - output_salidas
        output_delta = ErrorSalida * deriv_eval_vec(output_FUNC, output_salidas)
        hidden_delta2 = deriv_eval_vec(hidden_FUNC2, hidden_salidas2) * (W3.T @ output_delta)
        hidden_delta1 = deriv_eval_vec(hidden_FUNC1, hidden_salidas1) * (W2.T @ hidden_delta2)

        # Actualizar matrices de pesos
        W1 = W1 + learning_rate * (hidden_delta1 @ X[fila : fila + 1, :])
        X01 = X01 + learning_rate * hidden_delta1
        W2 = W2 + learning_rate * (hidden_delta2 @ hidden_salidas1.T)
        X02 = X02 + learning_rate * hidden_delta2
        W3 = W3 + learning_rate * (output_delta @ hidden_salidas2.T)
        X03 = X03 + learning_rate * output_delta

    # Forward propagation con pesos actualizados
    hidden_estimulos1 = W1 @ X.T + X01
    hidden_salidas1 = func_eval_vec(hidden_FUNC1, hidden_estimulos1)
    hidden_estimulos2 = W2 @ hidden_salidas1 + X02
    hidden_salidas2 = func_eval_vec(hidden_FUNC2, hidden_estimulos2)
    output_estimulos = W3 @ hidden_salidas2 + X03
    output_salidas = func_eval_vec(output_FUNC, output_estimulos)

    # Calcular el error promedio general de TODOS los X
    Error = np.mean((Y.T - output_salidas) ** 2)

    print("Epoch:", epoch)
    print("Error:", Error)

X_test = np.load("x_pca_test_reduc.npy")
nombres_test = np.load("nombres_test.npy")
y_test = LabelBinarizer().fit_transform(nombres_test)
nom = np.unique(nombres)
rta_ok = 0

for fila in range(X_test.shape[0]):
    # Realizar la predicción
    hidden_estimulos1_predict = W1 @ X_test[fila : fila + 1, :].T + X01
    hidden_salidas1_predict = func_eval_vec(hidden_FUNC1, hidden_estimulos1_predict)
    hidden_estimulos2_predict = W2 @ hidden_salidas1_predict + X02
    hidden_salidas2_predict = func_eval_vec(hidden_FUNC2, hidden_estimulos2_predict)
    output_estimulos_predict = W3 @ hidden_salidas2_predict + X03
    output_salidas_predict = func_eval_vec(output_FUNC, output_estimulos_predict)

    pos_max = np.argmax(output_salidas_predict)

    if nom[np.argmax(y_test[fila])] == nom[pos_max]:
        rta_predic = "Ok"
        rta_ok = rta_ok + 1
    else:
        rta_predic = ""

    print(
        "En la foto a predecir de:",
        nom[np.argmax(y_test[fila])],
        ", predijo:",
        nom[pos_max],
        rta_predic,
    )

print("La cantidad de predicciones correctas fue", rta_ok, "de", X_test.shape[0])


Initial error: 0.2684503334616096
Epoch: 1
Error: 0.04188303924672669
Epoch: 2
Error: 0.04165940362799924
Epoch: 3
Error: 0.04161729839230839
Epoch: 4
Error: 0.04159355236338489
Epoch: 5
Error: 0.041572361594850636
Epoch: 6
Error: 0.041551052370276356
Epoch: 7
Error: 0.041528455210097154
Epoch: 8
Error: 0.04150335953556075
Epoch: 9
Error: 0.04147448193034284
Epoch: 10
Error: 0.0414404254163958
Epoch: 11
Error: 0.041399522946378116
Epoch: 12
Error: 0.041349656676881376
Epoch: 13
Error: 0.041288109895606925
Epoch: 14
Error: 0.04121149717788381
Epoch: 15
Error: 0.041115775338520746
Epoch: 16
Error: 0.04099630089085427
Epoch: 17
Error: 0.04084804032234933
Epoch: 18
Error: 0.040665947987790246
Epoch: 19
Error: 0.04044507332357697
Epoch: 20
Error: 0.04018113373898307
Epoch: 21
Error: 0.039874530593032774
Epoch: 22
Error: 0.03953367365735343
Epoch: 23
Error: 0.0391671570119095
Epoch: 24
Error: 0.038777546697060114
Epoch: 25
Error: 0.03836562452357376
Epoch: 26
Error: 0.03792814295366803
Epoch

In [41]:
import pandas as pd

# Crear una lista para almacenar los resultados de cada predicción
resultados = []

for fila in range(X_test.shape[0]):
    # Realizar la predicción
    hidden_estimulos1_predict = W1 @ X_test[fila : fila + 1, :].T + X01
    hidden_salidas1_predict = func_eval_vec(hidden_FUNC1, hidden_estimulos1_predict)
    hidden_estimulos2_predict = W2 @ hidden_salidas1_predict + X02
    hidden_salidas2_predict = func_eval_vec(hidden_FUNC2, hidden_estimulos2_predict)
    output_estimulos_predict = W3 @ hidden_salidas2_predict + X03
    output_salidas_predict = func_eval_vec(output_FUNC, output_estimulos_predict)
    pos_max = np.argmax(output_salidas_predict)

    # Verificar si la predicción es correcta
    prediccion_correcta = nom[np.argmax(y_test[fila])] == nom[pos_max]
    
    # Agregar el resultado a la lista
    resultados.append({'Foto': fila+1,
                       'Clase Real': nom[np.argmax(y_test[fila])],
                       'Clase Predicha': nom[pos_max],
                       'Predicción Correcta': prediccion_correcta})

# Crear la tabla con los resultados
tabla_resultados = pd.DataFrame(resultados)

# Reemplazar los valores de las etiquetas en la columna "Clase Real"
nuevos_valores = {'ad': 'Andres', 'cp': 'Claudia','ec': 'Elemir','ep': 'Eduardo','fh': 'Fernanda','gf': 'Geronimo','hg': 'Hernan','ja': 'Julieta','jm': 'Jiang','jp': 'Josefina','jt': 'Joaquin','lm': 'Lujan','mf': 'Maribel','mg': 'Marcelo G.','mk': 'Marisa','ml': 'Maira','mt': 'Marcelo T.','nl': 'Nestor','ob': 'Oscar','rb': 'Ramon','re': 'Rodrigo','sp': 'Sebastian','ss': 'Silvia'}
tabla_resultados['Clase Real'].replace(nuevos_valores, inplace=True)
tabla_resultados['Clase Predicha'].replace(nuevos_valores, inplace=True)

# Mostrar la tabla
print(tabla_resultados)



    Foto  Clase Real Clase Predicha  Predicción Correcta
0      1      Andres         Andres                 True
1      2     Claudia       Geronimo                False
2      3      Elemir         Elemir                 True
3      4     Eduardo        Eduardo                 True
4      5    Fernanda       Fernanda                 True
5      6    Geronimo       Geronimo                 True
6      7      Hernan         Hernan                 True
7      8     Julieta        Maribel                False
8      9       Jiang          Jiang                 True
9     10    Josefina         Silvia                False
10    11     Joaquin        Joaquin                 True
11    12       Lujan          Lujan                 True
12    13     Maribel        Julieta                False
13    14  Marcelo G.     Marcelo G.                 True
14    15      Marisa         Silvia                False
15    16       Maira          Maira                 True
16    17  Marcelo T.     Marcel